#### Web Scrapping dados INMET

Existe um padrão dos links de downloads referenciados pelo elemento css "href" para executar o download do .zip com todas as informações do ano em questão.

Esse padrão contém os termos "dadoshistoricos" e um .zip referenciando o arquivo.

Vou acessar a página e extrair todos os conteúdos do elemento 'href' da página para fazer o download dos arquivos.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

from datetime import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By

from src.web_scrapping import browser
from src.web_scrapping import download
from src.data import etl

chromedriver_exe = '../../utils/chromedriver.exe'

In [ ]:
dir_download=r'C:\Users\breno\OneDrive\Documentos\Faculdade\FIAP\Challenge\2024\nimbus\data\raw\inmet_historico'

In [ ]:
# Inicia navegador
nav = browser.inicia_navegador(
    dir_download=dir_download,
    chromedriver_exe=chromedriver_exe
)

In [ ]:
# Acessa o site dos arquivos mensais
browser.acessa_pagina(
    navegador=nav,
    url="https://portal.inmet.gov.br/dadoshistoricos"
)

In [ ]:
# Aguarda até que o elemento com os downloads seja localizado
browser.aguarda_elemento(
    navegador=nav, 
    tempo=20, 
    localizador=(By.CLASS_NAME, "container")
)

In [ ]:
# Obtendo todos os elementos 'href'
href_page = nav.find_elements(By.CSS_SELECTOR, "a[href]")

# Extraindo o atributo (link) de cada um deles.
hrefs = [e.get_attribute("href") for e in href_page]

In [ ]:
# Links de download dos arquivos
link_download = [l for l in hrefs if "dadoshistoricos" in l and ".zip" in l]

In [ ]:
# Definindo um diretório para salvar o arquivo extraído
dir_ext = '../../data/processed/inmet_historico'

In [ ]:
# Para cada link, valida se já temos os dados do ano correspondente, avalia o download e descompacta
for link in link_download:
    ano = link[link.find(".zip")-4:link.find(".zip")]
    if download.verifica_existencia_arquivo(nome=ano, dir=dir_ext):
        pass
    else:
        dir_zip = f'../../data/raw/inmet_historico/{ano}.zip'
        url = link
        browser.acessa_pagina(nav, url)
        while not os.path.exists(dir_zip):
            time.sleep(2)
        extracao.descompacta_zip(dir_zip, dir_ext)

In [ ]:
# Encerrando o navegador
nav.quit()